In [9]:
%run utils.ipynb
#Leemos el dataset reducido
es_train, es_train, es_test, en_test,es_texts,en_texts = init_data()


#Ahora vamos a tokenizar los diferentes datasets para ello usaremos Keras y su Tokenizer
es_tok,en_tok,es_len,en_len,es_vocab,en_vocab = init_tokens(en_texts,es_texts)

print("Español- Numero de palabras: ", es_vocab, "\t Frase mas larga: ", es_len)
print("Ingles - Numero de palabras: ", en_vocab, "\t Frase mas larga: ", en_len)

#Dividimos el dataset en entrenamiento y prueba
es_train, es_test, en_train, en_test = split_dataset(es_texts,en_texts)

Español- Numero de palabras:  4050 	 Frase mas larga:  102
Ingles - Numero de palabras:  4841 	 Frase mas larga:  105


In [10]:
%run utils.ipynb
from tensorflow.keras import layers as layers
from tensorflow.keras.models import Model 
from tensorflow.keras.models import load_model

#Ahora vamos a modificar el modelo para que sea capaz de traducir una frase que nosotros le introduzcamos.
#Vamos a añadir recursividad al decoder
trained_model = load_model("gru_inference.h5")

#-----
#Encoder
#-----
#Es exactamente igual que el del anterior:
#Definimos la capa "input" para nuestro encoder
es_input_layer = layers.Input(shape=(es_len, es_vocab))

#Definimos la capa "Gru" para el encoder, tendremos que definir el tamaño (numero de neuronas)de la capa oculta 
#que hemos ido probando a base de prueba y error
hidden_units = 128
es_gru_layer = layers.GRU(hidden_units, return_state = True, weights = trained_model.get_layer(index=2).get_weights())

#Obtenemos la salida del encoder y el estado:
es_output, es_state = es_gru_layer(es_input_layer)

#Pero aqui creamos un modelo solo para el encoder
encoder = Model(inputs=es_input_layer, outputs = es_state)
encoder.summary()
#-----
#Decoder
#-----
#Es en el decoder donde vendrán las modificaciones
#Creamos 2 capas de entradas:
#La primera aceptara un solo vector onehot
en_input_layer = layers.Input(shape=(1,en_vocab))
#La otra aceptara una entrada para el estado t-1
en_state_in = layers.Input(shape=(hidden_units,))
#Definimos la capa Gru
en_gru_layer = layers.GRU(hidden_units, return_state = True, weights = trained_model.get_layer(index=3).get_weights())
#Obtenemos la salida y estado de la capa GRU
en_output, en_output_state = en_gru_layer(en_input_layer, initial_state = en_state_in)
#Definimos una ultima capa Dense
en_dense_layer = layers.Dense(en_vocab,activation="softmax",weights = trained_model.get_layer(index=4).get_weights())
en_prediction = en_dense_layer(en_output)

decoder = Model(inputs=[en_input_layer, en_state_in], outputs =[en_prediction, en_output_state])
decoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 102, 4050)]       0         
_________________________________________________________________
gru_5 (GRU)                  [(None, 128), (None, 128) 1605120   
Total params: 1,605,120
Trainable params: 1,605,120
Non-trainable params: 0
_________________________________________________________________
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 1, 4841)]    0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 128)]        0                                            
_

In [11]:
from tensorflow.keras import layers as layers
import tensorflow.keras.models as models
import numpy as np

def word_to_onehot(tokenizer, word, vocab_size):
    seq = tokenizer.texts_to_sequences([[word]])
    seq = to_categorical(seq, num_classes=vocab_size)
    seq = np.expand_dims(seq, axis=1)
    return seq

def probability_to_word(tokenizer, prob):
    word_index = np.argmax(prob[0,:], axis=-1)
    word = tokenizer.index_word[word_index]
    return word
#Ahora lo siguiente que tenemos que hacer es cargar el modelo entrenado ("Modelo anterior a este") y copiar
#los pesos
#En nuestro nuevo modelo hay 3 capas que contienen pesos, la Gru layer del encoder y decoder y la Dense layer del 
#decoder
#Leer archivo


#Introducimos una frase
es_sentence=["de acuerdo con lo que se. "]

es_seq = sentences_to_sequences(es_tok,es_sentence,es_len, es_vocab)
#Vamos a predecir el primer el estado a traves del encoder
en_state = encoder.predict(es_seq)
en_seq = word_to_onehot(en_tok, "sos", en_vocab)
en_sentence = ""
for i in range(en_len):
    en_probability, en_state = decoder.predict([en_seq,en_state])
    word = probability_to_word(en_tok,en_probability)
    en_seq = word_to_onehot(en_tok, word, en_vocab)
    if word == "eos": break
    en_sentence += word+" "
    print(word)
print(en_sentence)

KeyError: 0